In [54]:
%pip install scikit-learn optuna matplotlib seaborn pandas tables tabulate

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [55]:
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import matthews_corrcoef
import optuna
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tabulate import tabulate

# Data Load

In [6]:
from glob import glob
df = []
for path in glob("data/ID18150/Day2/DataFrame_Imaging_dFF_18150_day2_part_*.h5"):
    df.append(pd.read_hdf(path))
df = pd.concat(df)
# Get a list of columns whose names are of numerical type
numerical_columns = [col for col in df.columns if type(col) == int]

#Feature matrix
X = df[numerical_columns].values
t = df.Time.to_list()
y = df['Pump'].astype(int)
n_features = len(numerical_columns)

# Modelling

In [56]:
n_folds = 10
def objective(trial):
    kf = KFold(n_splits=n_folds)
    mcc_acc = []
    for split in kf.split(X):
        X_train, X_test = X[split[0]], X[split[1]]
        y_train, y_test = y[split[0]], y[split[1]]
        pca = PCA(n_components=trial.suggest_int('n_components', int(0.05*n_features), int(0.5*n_features)))
        pca.fit(X_train)
        X_train_pca = pca.transform(X_train)
        X_test_pca = pca.transform(X_test)
        clf = KNeighborsClassifier(n_neighbors=trial.suggest_int('n_neighbors', 3, 50), metric = 'cosine', n_jobs=-1)
        clf.fit(X_train_pca, y_train)
        y_pred = clf.predict(X_test_pca)
        mcc = matthews_corrcoef(y_test, y_pred)
        mcc_acc.append(mcc)
    hist = np.histogram(mcc_acc, bins=5)
    print(tabulate(hist))
    return np.mean(mcc_acc)

In [57]:
study = optuna.create_study(direction='maximize', storage="sqlite:///data/ID18150/Day2/pca_classifier.db", load_if_exists= True, study_name = "pca_optimization_cosine")
study.optimize(objective, n_trials=100)

[I 2024-11-07 01:46:30,409] A new study created in RDB with name: pca_optimization_cosine


[W 2024-11-07 01:48:45,588] Trial 0 failed with parameters: {'n_components': 310, 'n_neighbors': 45} because of the following error: TypeError("'module' object is not callable").
Traceback (most recent call last):
  File "/usr/local/python/3.10.13/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_2724/3042158684.py", line 18, in objective
    print(tabulate(hist))
TypeError: 'module' object is not callable
[W 2024-11-07 01:48:45,590] Trial 0 failed with value None.


TypeError: 'module' object is not callable